# SystemsAgeLung

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import sys
import subprocess
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
from pathlib import Path
import numpy as np
import torch.nn as nn

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.SystemsAgeLung)

class SystemsAgeLung(SystemsAgeBase):
    def __init__(self):
        super().__init__()
        self.prediction_index = 9



In [3]:
model = pya.models.SystemsAgeLung()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'systemsagelung'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2025
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Sehgal, Raghav, et al. \"Systems Age: A single blood methylation test to quantify aging heterogeneity across 11 physiological systems.\" Nature Aging (2025): 1-17."
model.metadata["doi"] = "https://doi.org/10.1038/s43587-025-00958-3"
model.metadata["research_only"] = True
model.metadata["notes"] = None

## Download clock dependencies

#### Download from R package

In [ ]:
FILE_ID = "14HBIhA-gkp9-RWD0HVAekRGWOHQBqvhc"
OUT_PATH = "SystemsAge_data.RData"

if not os.path.exists(OUT_PATH):
    try:
        import gdown  # type: ignore
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", "gdown"])
        import gdown  # type: ignore
    url = f"https://drive.google.com/uc?id={FILE_ID}"
    gdown.download(url, OUT_PATH, quiet=False)

print(f"Downloaded: {os.path.exists(OUT_PATH)} -> {OUT_PATH}")


Downloading...
From (original): https://drive.google.com/uc?id=14HBIhA-gkp9-RWD0HVAekRGWOHQBqvhc
From (redirected): https://drive.google.com/uc?id=14HBIhA-gkp9-RWD0HVAekRGWOHQBqvhc&confirm=t&uuid=77f24763-0c22-4b2b-b5dc-08d50fb4cc3b
To: /Users/lucascamillo/pyaging/clocks/notebooks/SystemsAge_data.RData
100%|██████████| 3.86G/3.86G [10:30<00:00, 6.13MB/s]

Downloaded: True -> SystemsAge_data.RData


In [ ]:
%%writefile export_systemsage.R
suppressPackageStartupMessages(library(jsonlite))
suppressPackageStartupMessages(library(readr))
suppressPackageStartupMessages(library(tibble))

use_arrow <- requireNamespace("arrow", quietly = TRUE)

write_matrix <- function(df, base_name) {
  if (use_arrow) {
    parquet_path <- paste0(base_name, ".parquet")
    written <- tryCatch({
      arrow::write_parquet(df, parquet_path, compression = "snappy")
      TRUE
    }, error = function(e) {
      message(sprintf("arrow parquet export failed for %s (%s); writing gzip CSV instead.", base_name, e$message))
      FALSE
    })
    if (isTRUE(written)) {
      return(parquet_path)
    }
    if (file.exists(parquet_path) && file.info(parquet_path)$size == 0) {
      unlink(parquet_path)
    }
  }
  csv_path <- paste0(base_name, ".csv.gz")
  readr::write_csv(df, csv_path)
  return(csv_path)
}

rds <- "SystemsAge_data.RData"
if (!file.exists(rds)) {
  stop(paste("Missing", rds, "- run the download cell first."))
}
load(rds)

# 1) CpG list used by the clock
CpGs <- rownames(DNAmPCA$rotation)
write_json(CpGs, "systemsage_CpGs.json", auto_unbox = TRUE)

# 2) Mean-imputation values for missing CpGs (named vector)
write_json(as.list(imputeMissingCpGs), "systemsage_impute_means.json", digits = 10, auto_unbox = TRUE)

# 3) DNAm PCA components: rotation matrix plus supporting statistics
rot_tbl <- as_tibble(DNAmPCA$rotation, rownames = "CpG")
rotation_path <- write_matrix(rot_tbl, "systemsage_dnam_pca_rotation")

write_json(as.list(DNAmPCA$center), "systemsage_dnam_pca_center.json", digits = 10, auto_unbox = TRUE)
write_json(DNAmPCA$scale, "systemsage_dnam_pca_scale.json", digits = 10, auto_unbox = TRUE)
write_json(as.numeric(DNAmPCA$sdev), "systemsage_dnam_pca_sdev.json", digits = 10, auto_unbox = TRUE)

# 4) Map DNAm PCs to system-level components
svc_tbl <- as_tibble(system_vector_coefficients, rownames = "PC")
system_vector_path <- write_matrix(svc_tbl, "systemsage_system_vector_coefficients")

write_json(as.list(system_scores_coefficients_scale), "systemsage_system_scores_coefficients_scale.json", digits = 10, auto_unbox = TRUE)

component_map <- tibble(
  component = colnames(system_vector_coefficients),
  system = gsub("[0-9]+$", "", component)
)
write_csv(component_map, "systemsage_system_component_map.csv")

# 5) Predicted chronological age model (DNAm PCs -> age before calibration)
pred_age <- list(
  intercept = as.numeric(Predicted_age_coefficients[1]),
  loadings = as.numeric(Predicted_age_coefficients[-1])
)
write_json(pred_age, "systemsage_predicted_age_linear.json", digits = 10, auto_unbox = TRUE)
write_json(list(a = Age_prediction_model[1], b = Age_prediction_model[2], c = Age_prediction_model[3]),
           "systemsage_predicted_age_poly.json", digits = 10, auto_unbox = TRUE)

# 6) PCA on system scores (11 systems + age prediction)
sys_rot_tbl <- as_tibble(systems_PCA$rotation, rownames = "variable")
systems_pca_rotation_path <- write_matrix(sys_rot_tbl, "systemsage_systems_pca_rotation")
write_json(as.list(systems_PCA$center), "systemsage_systems_pca_center.json", digits = 10, auto_unbox = TRUE)
write_json(as.list(systems_PCA$scale),  "systemsage_systems_pca_scale.json",  digits = 10, auto_unbox = TRUE)
write_json(as.numeric(systems_PCA$sdev), "systemsage_systems_pca_sdev.json", digits = 10, auto_unbox = TRUE)

# 7) Final SystemsAge coefficients (systems PCA -> SystemsAge)
write_json(as.numeric(Systems_clock_coefficients), "systemsage_final_coefficients.json", digits = 10, auto_unbox = TRUE)

# 8) Transformation coefficients for scaling outputs (11 systems + age prediction + SystemsAge)
system_labels <- c("Blood", "Brain", "Inflammation", "Heart", "Hormone", "Immune", "Kidney", "Liver", "Metabolic", "Lung", "MusculoSkeletal", "Age_prediction", "SystemsAge")
trans_tbl <- tibble(
  label = system_labels,
  Coef1 = transformation_coefs[, 1],
  Coef2 = transformation_coefs[, 2],
  Coef3 = transformation_coefs[, 3],
  Coef4 = transformation_coefs[, 4]
)
write_csv(trans_tbl, "systemsage_transformation_coeffs.csv")

# 9) Metadata summary
meta <- list(
  n_cpgs = length(CpGs),
  n_dnam_pcs = ncol(DNAmPCA$rotation),
  n_system_components = ncol(system_vector_coefficients),
  n_systems = length(system_labels),
  files = list(
    dnam_pca_rotation = rotation_path,
    system_vector_coefficients = system_vector_path,
    systems_pca_rotation = systems_pca_rotation_path
  )
)
write_json(meta, "systemsage_export_summary.json", auto_unbox = TRUE)

cat("Export complete\n")



In [ ]:
os.system("Rscript export_systemsage.R")

## Load features

#### From exported assets


In [5]:
with open('systemsage_CpGs.json', 'r') as f:
    model.features = json.load(f)

## Load weights into base model

#### From JSON file

In [6]:
with open('systemsage_predicted_age_linear.json', 'r') as f:
    predicted_age_linear = json.load(f)

with open('systemsage_predicted_age_poly.json', 'r') as f:
    predicted_age_poly = json.load(f)

with open('systemsage_system_scores_coefficients_scale.json', 'r') as f:
    system_scores_coeff_scale = json.load(f)

with open('systemsage_systems_pca_center.json', 'r') as f:
    systems_pca_center = json.load(f)

with open('systemsage_systems_pca_scale.json', 'r') as f:
    systems_pca_scale = json.load(f)

with open('systemsage_dnam_pca_center.json', 'r') as f:
    dnam_pca_center = json.load(f)

with open('systemsage_dnam_pca_scale.json', 'r') as f:
    dnam_pca_scale = json.load(f)

with open('systemsage_final_coefficients.json', 'r') as f:
    systemsage_final_coeffs = json.load(f)

transformation_coeffs = pd.read_csv('systemsage_transformation_coeffs.csv')
system_component_map = pd.read_csv('systemsage_system_component_map.csv')
systems_pca_asset = Path('systemsage_systems_pca_rotation.csv.gz')
rotation_asset = Path('systemsage_dnam_pca_rotation.csv.gz')
system_vector_asset = Path('systemsage_system_vector_coefficients.csv.gz')

with open('systemsage_export_summary.json', 'r') as f:
    export_summary = json.load(f)

export_summary


{'n_cpgs': 125175,
 'n_dnam_pcs': 4018,
 'n_system_components': 84,
 'n_systems': 13,
 'files': {'dnam_pca_rotation': 'systemsage_dnam_pca_rotation.csv.gz',
  'system_vector_coefficients': 'systemsage_system_vector_coefficients.csv.gz',
  'systems_pca_rotation': 'systemsage_systems_pca_rotation.csv.gz'}}

#### Build SystemsAge base model


In [8]:
# DNAm PCA rotation and centering
rotation_df = pd.read_csv(rotation_asset, compression='gzip')
if rotation_df['CpG'].tolist() != model.features:
    raise ValueError('Feature order mismatch between rotation matrix and model features')
model.dnam_rotation = torch.tensor(rotation_df.drop(columns=['CpG']).to_numpy(dtype=np.float32))
model.dnam_center = torch.tensor([dnam_pca_center[cpg] for cpg in model.features], dtype=torch.float32)
del rotation_df

# System vector coefficients (DNAm PCs -> system components)
system_vector_df = pd.read_csv(system_vector_asset, compression='gzip')
component_names = system_vector_df.columns.tolist()[1:]
model.system_vector = torch.tensor(
    system_vector_df.drop(columns=['PC']).fillna(0.0).to_numpy(dtype=np.float32)
)
del system_vector_df
component_index = {name: idx for idx, name in enumerate(component_names)}

system_label_map = {'Inflammation': 'Cytokine'}
system_labels = transformation_coeffs['label'].tolist()[:11]
model.system_labels = system_labels

system_indices = []
system_modules = []
for label in system_labels:
    lookup = system_label_map.get(label, label)
    components = system_component_map.loc[
        system_component_map['system'] == lookup, 'component'
    ].tolist()
    if not components:
        raise ValueError(f'No components found for system {label}')

    indices = torch.tensor([component_index[c] for c in components], dtype=torch.long)
    weights = torch.tensor([system_scores_coeff_scale[c] for c in components], dtype=torch.float32)

    if weights.numel() == 1:
        weights = torch.tensor([-1.0], dtype=torch.float32)

    linear = pya.models.LinearModel(weights.numel())
    linear.linear.weight.data.zero_()
    linear.linear.bias.data.zero_()
    linear.linear.weight.data.copy_(weights.view(1, -1))
    linear.linear.bias.requires_grad_(False)
    linear.linear.weight.requires_grad_(False)

    system_indices.append(indices)
    system_modules.append(linear)

model.system_component_indices = system_indices
model.system_modules = nn.ModuleList(system_modules)

# Predicted chronological age assets
predicted_age_weights = torch.tensor(predicted_age_linear['loadings'], dtype=torch.float32)
predicted_age_model = pya.models.LinearModel(predicted_age_weights.numel())
predicted_age_model.linear.weight.data.zero_()
predicted_age_model.linear.bias.data.zero_()
predicted_age_model.linear.weight.data.copy_(predicted_age_weights.unsqueeze(0))
predicted_age_model.linear.bias.data.fill_(float(predicted_age_linear['intercept']))
for param in predicted_age_model.parameters():
    param.requires_grad = False
model.predicted_age_model = predicted_age_model
model.predicted_age_poly = torch.tensor(
    [predicted_age_poly['a'], predicted_age_poly['b'], predicted_age_poly['c']],
    dtype=torch.float32,
)

# Systems PCA assets (system scores + age prediction)
systems_pca_order = transformation_coeffs['label'].tolist()[:12]
systems_pca_df = pd.read_csv(systems_pca_asset, compression='gzip').set_index('variable')
rotation_cols = [col for col in systems_pca_df.columns if col.startswith('PC')]
scale_vec = torch.tensor([systems_pca_scale[label] for label in systems_pca_order], dtype=torch.float32)
center_vec = torch.tensor([systems_pca_center[label] for label in systems_pca_order], dtype=torch.float32)
rotation_mat = torch.tensor(
    systems_pca_df.loc[systems_pca_order, rotation_cols].to_numpy(dtype=np.float32)
)
rotation_scaled = rotation_mat / scale_vec.unsqueeze(1)

systems_pca_model = pya.models.PCLinearModel(input_dim=len(systems_pca_order), pc_dim=rotation_scaled.size(1))
systems_pca_model.center.data.copy_(center_vec)
systems_pca_model.rotation.data.copy_(rotation_scaled)
systems_pca_model.linear.weight.data.zero_()
systems_pca_model.linear.bias.data.zero_()
systems_pca_model.linear.weight.data.copy_(
    torch.tensor(systemsage_final_coeffs, dtype=torch.float32).view(1, -1)
)
systems_pca_model.linear.bias.requires_grad_(False)
systems_pca_model.linear.weight.requires_grad_(False)
for param in systems_pca_model.parameters():
    param.requires_grad = False
model.systems_pca_model = systems_pca_model

# Transformation assets
model.transformation_coefs = torch.tensor(
    transformation_coeffs[['Coef1', 'Coef2', 'Coef3', 'Coef4']].to_numpy(dtype=np.float32)
)
model.transformation_labels = transformation_coeffs['label'].tolist()

# Record summary metadata for downstream reference
model.export_summary = export_summary

#### From JSON file

In [9]:
with open('systemsage_impute_means.json', 'r') as f:
    reference_feature_values = json.load(f)
model.reference_values = torch.tensor(
    [reference_feature_values[cpg] for cpg in model.features], dtype=torch.float32
)

## Load preprocess and postprocess objects

In [10]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [11]:
model.postprocess_name = None
model.postprocess_dependencies = None


In [12]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Sehgal, Raghav, et al. "Systems Age: A single blood methylation '
             'test to quantify aging heterogeneity across 11 physiological '
             'systems." Nature Aging (2025): 1-17.',
 'clock_name': 'systemsagelung',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1038/s43587-025-00958-3',
 'notes': None,
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2025}
reference_values: [0.7703027725219727, 0.10787541419267654, 0.14593413472175598, 0.15467368066310883, 0.05177399888634682, 0.7689204812049866, 0.09734559804201126, 0.5384307503700256, 0.03857552260160446, 0.14853332936763763, 0.03710409998893738, 0.7541770339012146, 0.05995236337184906, 0.7082317471504211, 0.9218639731407166, 0.04736267030239105, 0.11038260161876678, 0.16661499440670013, 0.08505702018737793

## Basic test

In [13]:
torch.manual_seed(42)
input_sample = torch.randn(1, len(model.features), dtype=torch.float32)
model.eval()
model.to(torch.float32)
pred = model(input_sample)
pred


tensor([[170.6917]])

## Save torch model

In [14]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [ ]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)